Reference: https://github.com/matterport/Mask_RCNN/blob/master/samples/coco/coco.py

### Imports

In [ ]:
import os
import os.path
from os import path
import sys
import cv2
import random
import math
import json
import re
import time
import skimage.draw
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
from google.colab import drive
drive.mount('/content/drive')

# Root directory of the project
ROOT_DIR = os.path.abspath("/content/drive/My Drive/TCC")
MODA_DIR = os.path.join(ROOT_DIR, 'modanet/modanet/annotations/')
MODA_SEG_DIR = os.path.join(ROOT_DIR, 'Visualização/mascaras/test')
RCNN_DIR = os.path.join(ROOT_DIR, "Mask_RCNN-master")
# Import Mask RCNN from local folder
sys.path.append(RCNN_DIR)  

from mrcnn.config import Config
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline 

CUSTOM_DIR = os.path.join('/content/drive/My Drive/UFRN/FASHION/')
# Set path to balloon weights file
weights_path = os.path.join(ROOT_DIR, 'Mask_RCNN-master/mask_rcnn_coco.h5')
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "Train/logs/inspect")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Using TensorFlow backend.


## Functions

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax


def plot_figures(figures, nrows = 1, ncols=1):
    """Plot a dictionary of figures.

    Parameters
    ----------
    figures : <title, figure> dictionary
    ncols : number of columns of subplots wanted in the display
    nrows : number of rows of subplots wanted in the figure
    """

    fig, axeslist = plt.subplots(ncols=ncols, nrows=nrows)
    for ind,title in zip(range(len(figures)), figures):
        axeslist.ravel()[ind].imshow(figures[title], cmap=plt.jet())
        axeslist.ravel()[ind].set_axis_off()
    plt.tight_layout() # optional


def make_segmentation_mask(image, mask,rois):
        img = image.copy()
        img[:,:,0] *= mask
        img[:,:,1] *= mask
        img[:,:,2] *= mask
        img[img[:,:,:]== 0] = 255
        cropped = img[rois[0]:rois[2],rois[1]:rois[3]]

        return  cropped

## Mask R-CNN

### Config

In [ ]:
class modanetConfig(Config):
    """Configuration for training on MS COCO.
    Derives from the base Config class and overrides values specific
    to the COCO dataset.
    """
    # Give the configuration a recognizable name
    NAME = "coco"
    IMAGE_MIN_DIM = 400
    IMAGE_MAX_DIM = 1024
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.99
    # Number of classes (including background)
    NUM_CLASSES = 1 + 80  # COCO has 80 classes    

### Modanet Dataset

In [ ]:
class ModaNetDataset(utils.Dataset):
    """Generates the modanet synthetic dataset. """
    
    def load_moda(self, dataset_dir, subset):
        '''
        Loads a subset of the Paperdoll dataset, reads the JSON file,
        extracts the annotations and iteratively calls the internal add_class 
        and add_image functions to build the dataset for the model.
        
        Args:
          dataset_dir: Root directory of the dataset.
          subset: Subset to load: train or val
        '''
        self.add_class("coco", 1, "person")
        self.add_class("coco", 2, "bicycle")
        self.add_class("coco", 3, "car")
        self.add_class("coco", 4, "motorcycle")
        self.add_class("coco", 5, "airplane")
        self.add_class("coco", 6, "bus")
        self.add_class("coco", 7, "train")
        self.add_class("coco", 8, "truck")
        self.add_class("coco", 9, "boat")
        self.add_class("coco", 10, "traffic light")
        self.add_class("coco", 11, "fire hydrant")
        self.add_class("coco", 12, "stop sign")
        self.add_class("coco", 13, "parking meter")
        self.add_class("coco", 14, "bench")
        self.add_class("coco", 15, "bird")
        self.add_class("coco", 16, "cat")
        self.add_class("coco", 17, "dog")
        self.add_class("coco", 18, "horse")
        self.add_class("coco", 19, "boat")
        self.add_class("coco", 20, "sheep")
        self.add_class("coco", 21, "cow")
        self.add_class("coco", 22, "elepahant")
        self.add_class("coco", 23, "bear")
        self.add_class("coco", 24, "zebra")
        self.add_class("coco", 25, "giraffe")
        self.add_class("coco", 26, "backpack")
        self.add_class("coco", 27, "umbrella")
        self.add_class("coco", 28, "handbag")
        self.add_class("coco", 29, "tie")
        self.add_class("coco", 30, "suitcase")
        self.add_class("coco", 31, "frisbee")
        self.add_class("coco", 32, "skis")
        self.add_class("coco", 33, "snowboard")
        self.add_class("coco", 34, "sports ball")
        self.add_class("coco", 35, "kite")
        self.add_class("coco", 36, "baseball bat")
        self.add_class("coco", 37, "baseball glove")
        self.add_class("coco", 38, "skateboard")
        self.add_class("coco", 39, "surfboard")
        self.add_class("coco", 40, "tennis racket")
        self.add_class("coco", 41, "bottle")
        self.add_class("coco", 42, "wine glass")
        self.add_class("coco", 43, "cup")
        self.add_class("coco", 44, "fork")
        self.add_class("coco", 45, "knife")
        self.add_class("coco", 46, "spoon")
        self.add_class("coco", 47, "bowl")
        self.add_class("coco", 48, "banana")
        self.add_class("coco", 49, "apple")
        self.add_class("coco", 50, "sandwich")
        self.add_class("coco", 51, "orange")
        self.add_class("coco", 52, "broccoli")
        self.add_class("coco", 53, "carrot")
        self.add_class("coco", 54, "hot dog")
        self.add_class("coco", 55, "pizza")
        self.add_class("coco", 56, "donut")
        self.add_class("coco", 57, "cake")
        self.add_class("coco", 58, "chair")
        self.add_class("coco", 59, "couch")
        self.add_class("coco", 60, "potted plant")
        self.add_class("coco", 61, "bed")
        self.add_class("coco", 62, "dining table")
        self.add_class("coco", 63, "toilet")
        self.add_class("coco", 64, "tv")
        self.add_class("coco", 65, "laptop")
        self.add_class("coco", 66, "mouse")
        self.add_class("coco", 67, "remote")
        self.add_class("coco", 68, "keyboard")
        self.add_class("coco", 69, "cell phone")
        self.add_class("coco", 70, "microwave")
        self.add_class("coco", 71, "over")
        self.add_class("coco", 72, "toaster")
        self.add_class("coco", 73, "sink")
        self.add_class("coco", 74, "refrigerator")
        self.add_class("coco", 75, "book")
        self.add_class("coco", 76, "clock")
        self.add_class("coco", 77, "vase")
        self.add_class("coco", 78, "scissors")
        self.add_class("coco", 79, "teddy bear")
        self.add_class("coco", 80, "hair drier")
        self.add_class("coco", 81, "toothbrush")
        
        assert subset in ["train", "test", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        jsonall = json.load(open(os.path.join(dataset_dir, "instances.json")))
        values = list(jsonall.values())  
        class_nums = []
        polygons = []
        image_ids = []
        category = []
        polygons_list = []
        same_img = []
        localid=0
        cont_same = 0
        
        # go through annotations JSON to get the needed values to apply on add_image()
        for a in values[2]:
            polygons.append(a['segmentation']) 
            category.append(a['category_id']) 
            if localid != a['image_id']: 
                image_ids.append(a['image_id']) 
                localid = a['image_id']
                same_img.append(cont_same)
                cont_same = 1
            else:
                cont_same +=1
                        
        same_img.append(cont_same)
        same_img.pop(0)
        
        for i in range(len(image_ids)):
          path = image_ids[i]          
          image_path = os.path.join(dataset_dir, str(path) + '.jpg')
          for j in range(same_img[i]):
            class_nums.append(category.pop(0))
            polygons_list.append(polygons.pop(0))              
          self.add_image('moda',
                          image_id = path,
                          path = image_path,
                          width = 400,
                          height = 600,
                          polygons = polygons_list,
                          class_list = np.array(class_nums))
          class_nums=[]
          polygons_list=[]

          
    def load_mask(self, image_id):
        """Generate instance masks for an image for every object in the image
           by drawing the polygons.
           
           Args:
            image_id: the image loaded and annotation loaded by load_moda
           Returns:
               masks: A bool array of shape [height, width, instance count] with
                      one mask per instance.
               class_ids: a 1D array of class IDs of the instance masks.
        """
        class_ids = []
        polyx = []
        polyy = []
        polygon=[]
        x=[]
        y=[]
        cont = 0
        cont2=0
        cont3=0
        info = self.image_info[image_id]
        
        for value in info["polygons"]:
          if (len(value) < 2):
            for number in value:
              for item in number:
                if cont%2 == 0:
                  x.append(item)
                else:
                  y.append(item)
                cont +=1
            polyx.append(x)
            x=[]
            polyy.append(y)
            y=[]
          else:
            for valor in value[0]:
              if cont2%2 == 0:
                x.append(valor)
              else:
                y.append(valor)
              cont2+=1
            for valor2 in value[1]:
              if cont3%2 == 0:
                x.append(valor2)
              else:
                y.append(valor2)
              cont3+=1
            polyx.append(x)
            x=[]
            polyy.append(y)
            y=[]
        
        mask = np.zeros([info["height"], info["width"], len(polyx)], dtype=np.uint8)
        
        for i, p in enumerate(polyx):
          rr, cc = skimage.draw.polygon(polyy[i], polyx[i]) 
          mask[rr, cc, i] = 1
        class_array = info['class_list']
        return mask.astype(np.bool), class_array


    def image_reference(self, image_id):
        """
        Return the path of the image: a string that identifies the image
        for debugging purposes
        """
        info = self.image_info[image_id]
        if info["source"] == "moda":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)     

###Eval

In [ ]:
dataset = ModaNetDataset()
dataset.load_moda(MODA_DIR, "test")
dataset.prepare()
#print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

In [ ]:
config = modanetConfig()
#config.display()

In [ ]:
model = modellib.MaskRCNN(mode="inference",
                          model_dir = MODEL_DIR,
                          config = config)
# Load weights
# utils.download_trained_weights(COCO_MODEL_PATH)
weights_path = os.path.join(ROOT_DIR, 'mask_rcnn_coco.h5')
model.load_weights(weights_path, by_name=True)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.








In [ ]:
'''
image_id = random.choice(dataset.image_ids)
image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset,
                                                                          config,
                                                                          image_id,
                                                                          use_mini_mask=False)
info = dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"],
                                       info["id"],
                                       image_id,
                                       dataset.image_reference(image_id)))
                        

#img_list = ['49.jpg', '35.jpg', '117.jpg', '127.jpg']

#img_list = np.random.choice(dataset.image_ids, 3)
#MODA_TEST = os.path.join(MODA_DIR, "test/")
FASHION_TEST = os.path.join(ROOT_DIR, "fashion/")

for img in os.listdir(FASHION_TEST):
#for img in img_list:
    # n = dataset.image_reference(img)
    # print(n)
    # image = mpimg.imread(os.path.join(MODA_TEST, str(n)))
    image = mpimg.imread(os.path.join(FASHION_TEST, img))

    # Run object detection
    results = model.detect([image], verbose=1)

    # Display results
    ax = get_ax(1)
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                dataset.class_names, r['scores'], ax=ax, title="")
'''               

###Create Mask

In [ ]:
#img_list = np.random.choice(dataset.image_ids, 200)        
#img_list = ['49.jpg', '35.jpg', '117.jpg', '127.jpg']
FASHION_TEST = os.path.join(ROOT_DIR, "fashion/")
#for img in img_list:
for img in os.listdir(FASHION_TEST):
    #n = dataset.image_reference(img)
    #image = mpimg.imread(os.path.join(CUSTOM_DIR, str(n)))
    cont=0
    if ~path.exists('/content/drive/My Drive/TCC/Visualização/mascaras/moda/pessoa/'+str(img)+'.png'):
      image = mpimg.imread(os.path.join(FASHION_TEST, img))
      #image = mpimg.imread(str(n))
      # Run object detection
      results = model.detect([image], verbose=1)
      # Display results
      ax = get_ax(1)
      r = results[0]
      class_ids = r['class_ids']   
      figures ={}
      for i in range(r['masks'].shape[-1]):
          mask = r['masks'][:,:,i]
          croped_roi_image = make_segmentation_mask(image, mask, r['rois'][i])
          figures[i] = croped_roi_image
          mpimg.imsave('/content/drive/My Drive/TCC/Visualização/mascaras/moda/pessoa/'+str(img)+str(cont)+'.png', figures[i])        
          cont+=1
      plot_figures(figures, 1, 6)
      visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], dataset.class_names, r['scores'], ax=ax, title="")

Output hidden; open in https://colab.research.google.com to view.